In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2
import os
import torch
import torchvision
import torchvision.transforms.functional as fn
from PIL import Image
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from models.models import LPCNNv1

from src.data.car_plate_dataset import UFPRDataset, id_to_filepath

C:\Users\Arya\anaconda3\envs\vision\lib\site-packages\pkg_resources\__init__.py:123: PkgResourcesDeprecationWarning: -PKG-VERSION is an invalid version and will not be supported in a future release
  warnings.warn(


In [2]:
os.chdir(r"C:/Users/Arya/workspace/ProjectSentry/data/raw/UFPR-ALPR dataset/testing/track0091")
os.getcwd()

'C:\\Users\\Arya\\workspace\\ProjectSentry\\data\\raw\\UFPR-ALPR dataset\\testing\\track0091'

In [14]:
train_dataset = UFPRDataset(
    "C:\\Users\\Arya\\workspace\\ProjectSentry\\data\\raw\\UFPR-ALPR dataset\\training",
    resize=torchvision.transforms.Resize(size=500),
    grayscale=torchvision.transforms.Grayscale()
)

finished setting up data


In [99]:
# train_dataset.labels.items()
# train_dataset.__getitem__(29)
# train_dataset.labels["000130"][1]

array([1152,  506, 1209,  531])

In [ ]:
train_dataloader = DataLoader(train_dataset)
trainer = pl.Trainer(max_epochs=1)
model = LPCNNv1()

trainer.fit(model, train_dataloaders=train_dataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: C:\Users\Arya\workspace\ProjectSentry\data\raw\UFPR-ALPR dataset\training\track0028\lightning_logs

  | Name         | Type       | Params
--------------------------------------------
0 | conv1        | Conv2d     | 832   
1 | rel1         | ReLU       | 0     
2 | conv2        | Conv2d     | 51.3 K
3 | rel2         | ReLU       | 0     
4 | pool         | MaxPool2d  | 0     
5 | flatten      | Flatten    | 0     
6 | dense        | Linear     | 13.9 M
7 | rel3         | ReLU       | 0     
8 | bounding_box | Sequential | 772   
--------------------------------------------
13.9 M    Trainable params
0         Non-trainable params
13.9 M    Total params
55.631    Total estimated model params size (MB)
C:\Users\Arya\anaconda3\envs\vision\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: Possib

Training: 0it [00:00, ?it/s]

torch.Size([1, 500, 888])
tensor([-0.0170,  0.0270,  0.0136, -0.0617], grad_fn=<MeanBackward1>)
tensor([[421, 242, 455, 257]], dtype=torch.int32)
tensor(343.7595, grad_fn=<MeanBackward0>)


C:\Users\Arya\workspace\ProjectSentry\models\models.py:83: UserWarning: Using a target size (torch.Size([1, 4])) that is different to the input size (torch.Size([4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.l1_loss(predicted_bb, target_bb[1])


torch.Size([1, 500, 888])
tensor([0.0726, 0.1003, 0.0939, 0.0075], grad_fn=<MeanBackward1>)
tensor([[423, 242, 457, 256]], dtype=torch.int32)
tensor(344.4314, grad_fn=<MeanBackward0>)
torch.Size([1, 500, 888])
tensor([0.2295, 0.2470, 0.2482, 0.1496], grad_fn=<MeanBackward1>)
tensor([[425, 241, 460, 255]], dtype=torch.int32)
tensor(345.0314, grad_fn=<MeanBackward0>)
torch.Size([1, 500, 888])
tensor([0.4680, 0.4738, 0.4821, 0.3712], grad_fn=<MeanBackward1>)
tensor([[428, 240, 462, 254]], dtype=torch.int32)
tensor(345.5512, grad_fn=<MeanBackward0>)
torch.Size([1, 500, 888])
tensor([0.7943, 0.7869, 0.8025, 0.6797], grad_fn=<MeanBackward1>)
tensor([[430, 240, 464, 254]], dtype=torch.int32)
tensor(346.2341, grad_fn=<MeanBackward0>)
torch.Size([1, 500, 888])
tensor([1.2130, 1.1915, 1.2145, 1.0798], grad_fn=<MeanBackward1>)
tensor([[432, 240, 466, 254]], dtype=torch.int32)
tensor(346.8253, grad_fn=<MeanBackward0>)
torch.Size([1, 500, 888])
tensor([1.7283, 1.6924, 1.7226, 1.5764], grad_fn=<Mean

In [35]:
# model = LPCNNv1()
# model.load_from_checkpoint(r"C:\Users\Arya\workspace\ProjectSentry\data\raw\UFPR-ALPR dataset\training\lightning_logs\version_48\checkpoints\epoch=0-step=1800.ckpt")


LPCNNv1(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (rel1): ReLU()
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (rel2): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (dense): Linear(in_features=108240, out_features=128, bias=True)
  (rel3): ReLU()
  (bounding_box): Sequential(
    (0): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Linear(in_features=128, out_features=4, bias=True)
  )
)

In [39]:
os.chdir(r"C:/Users/Arya/workspace/ProjectSentry/data/raw/UFPR-ALPR dataset/training/track0028")
resize=torchvision.transforms.Resize(size=500)
grayscale=torchvision.transforms.Grayscale()
plt.close()
fig, ax = plt.subplots()
image = Image.open("track0028[01].png")
image = Image.fromarray(np.asarray(resize(image)))
image = Image.fromarray(np.asarray(grayscale(image)))
ax.imshow(image, cmap='gray')

image = torch.tensor(np.asarray(image), dtype=torch.float32)
bb = model(image[None, :]).detach().mean(axis=0)
print(bb)
x_min, y_min, x_max, y_max = bb
rect = patches.Rectangle((x_min, y_min),x_max-x_min, y_max-y_min, linewidth=.5, edgecolor='r', facecolor='none' )
ax.add_patch(rect)

plt.show()

C:\Users\Arya\AppData\Local\Temp\ipykernel_19740\1225892266.py:4: MatplotlibDeprecationWarning: The close_event function was deprecated in Matplotlib 3.6 and will be removed two minor releases later. Use callbacks.process('close_event', CloseEvent(...)) instead.
  plt.close()


<IPython.core.display.Javascript object>

torch.Size([1, 500, 888])
tensor([380.0337, 227.2269, 413.6375, 236.2407])
